SPRINT 8 - MACHINE LEARNING
--
Descripcion.

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Se trabaja con datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. 

Buscaremos un umbral de exactitud del 0.75. 

Preparacion de datos y herramientas de trabajo.
---

In [1]:
#IMPORTAR LIBRERIAS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('dataset/users_behavior.csv')
df.info()
print(df.head(10))

print('\nPodemos ver que tenemos ',df.duplicated().sum(),'duplicados')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
5   58.0   344.56      21.0  15823.37         0
6   57.0   431.64      20.0   3738.90         1
7   15.0   132.40       6.0  21911.60         0
8    7.0    43.39       3.0   2538.67         1
9   90.0   665.41      38.0  17358.61         0

Podemos ver que tenemos  0 duplicados


In [3]:
print(df.groupby('is_ultra').sum())
print(df.groupby('is_ultra').count())

             calls    minutes  messages      mb_used
is_ultra                                            
0         130315.0  904846.84   74413.0  36128672.83
1          72292.0  503556.20   48623.0  19176790.88
          calls  minutes  messages  mb_used
is_ultra                                   
0          2229     2229      2229     2229
1           985      985       985      985


Podemos ver que tenemos mas clientes utilizando el plan SMART que el plan ULTRA. 
Por esto, la cantidad de mensajes, llamadas y de uso de datos será mayor en el plan SMART. 
Tambien podemos ver con ayuda de info() que no contamos con valores nulos y de igual forma no tenemos valores duplicados.

Dividir el dataset en un conjunto de entrenamiento y en un conjunto de validacion con un porcentaje del 25% y un valor de estado random de 54321. 
Tendremos 2410 filas en el conjunto de entrenamiento y 804 filas en el conjunto de validacion. 

In [4]:
#Dividir el data set en validacion y entrenamiento.
df_train, df_valid = train_test_split(df, test_size=0.25, random_state=54321)
df_train.info()
print(df_train.head())
print()
df_valid.info()
print(df_valid.head())

<class 'pandas.core.frame.DataFrame'>
Index: 2410 entries, 103 to 2641
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     2410 non-null   float64
 1   minutes   2410 non-null   float64
 2   messages  2410 non-null   float64
 3   mb_used   2410 non-null   float64
 4   is_ultra  2410 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 113.0 KB
      calls  minutes  messages   mb_used  is_ultra
103    74.0   540.81      17.0  20075.65         0
1695   62.0   473.14       4.0  14623.29         1
1542   66.0   430.02      20.0  12049.53         0
636    92.0   628.47      72.0  16839.52         0
1216   44.0   318.99      48.0  14817.03         0

<class 'pandas.core.frame.DataFrame'>
Index: 804 entries, 1727 to 720
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     804 non-null    float64
 1   minutes   804 non-null    float64
 2   me

Generar 4 diferentes datasets. 
- 2 datasets con los datos de entrenamiento. Un dataset con la columna objetivo de is_ultra y otro dataset con las demas columnnas para formar el dataset de caracteristicas. 
- 2 datasets con los datos de validacion. Un dataset con la columna objetivo de is_ultra y otro dataset con las demas columnnas para formar el dataset de caracteristicas. 

In [5]:
#CARACTERISTICAS Y OBJETIVO DE ENTRENAMIENTO
features_train = df_train.drop('is_ultra',axis=1)
target_train = df_train['is_ultra']
features_train.info()
print()
target_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2410 entries, 103 to 2641
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     2410 non-null   float64
 1   minutes   2410 non-null   float64
 2   messages  2410 non-null   float64
 3   mb_used   2410 non-null   float64
dtypes: float64(4)
memory usage: 94.1 KB

<class 'pandas.core.series.Series'>
Index: 2410 entries, 103 to 2641
Series name: is_ultra
Non-Null Count  Dtype
--------------  -----
2410 non-null   int64
dtypes: int64(1)
memory usage: 37.7 KB


In [6]:
#CARACTERISTICAS Y OBJETIVO DE VALIDACION
features_valid = df_valid.drop('is_ultra',axis=1)
target_valid = df_valid['is_ultra']
features_valid.info()
print()
target_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 804 entries, 1727 to 720
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     804 non-null    float64
 1   minutes   804 non-null    float64
 2   messages  804 non-null    float64
 3   mb_used   804 non-null    float64
dtypes: float64(4)
memory usage: 31.4 KB

<class 'pandas.core.series.Series'>
Index: 804 entries, 1727 to 720
Series name: is_ultra
Non-Null Count  Dtype
--------------  -----
804 non-null    int64
dtypes: int64(1)
memory usage: 12.6 KB


Decision Tree Classifier Model. 
---
Random_state de 54321.
Primero calcular la exactitud con una max_depth predeterminado. 
Segundo revisar que exactitud es la mejor modificando el max_depth, usaremos un rango de 1 a 10.

In [7]:
#modelo de clasificacion de arbol
model_tree = DecisionTreeClassifier(random_state=54321)
model_tree.fit(features_train,target_train)

#Prediccion del modelo de clasificacion con los valores de validacion
valid_tree_prediction = model_tree.predict(features_valid)
print('Exactitud del conjunto de validacion:', accuracy_score(target_valid,valid_tree_prediction))

Exactitud del conjunto de validacion: 0.7263681592039801


In [8]:
#DECISION TREE CLASSIFIER DIFERENTES PROFUNDIDADES
best_score_tree = 0
for depth in range(1,11):
    model_tree = DecisionTreeClassifier(random_state=54321 , max_depth= depth)
    model_tree.fit(features_train , target_train)
    valid_tree_prediction = model_tree.predict(features_valid)
    print('max_depth = ' , depth , ': ' , end= '')
    score_tree = accuracy_score(target_valid,valid_tree_prediction)
    print(score_tree)
    if score_tree > best_score_tree:
        best_score_tree = score_tree
        best_max_depth = depth
        best_model_tree = model_tree
        best_valid_tree_prediction = valid_tree_prediction

print('\nEl modelo con la mejor exactitud ', best_score_tree,'con un max_depth = ',best_max_depth,'\nel modelo fue:',best_model_tree)
#print(valid_tree_prediction)


max_depth =  1 : 0.736318407960199
max_depth =  2 : 0.7699004975124378
max_depth =  3 : 0.7810945273631841
max_depth =  4 : 0.7810945273631841
max_depth =  5 : 0.7810945273631841
max_depth =  6 : 0.7835820895522388
max_depth =  7 : 0.7873134328358209
max_depth =  8 : 0.7947761194029851
max_depth =  9 : 0.7922885572139303
max_depth =  10 : 0.7910447761194029

El modelo con la mejor exactitud  0.7947761194029851 con un max_depth =  8 
el modelo fue: DecisionTreeClassifier(max_depth=8, random_state=54321)


Con el modelo DecisionTreeClassifier se obtuvo una exactitud del 0.79 con la profundidad de 8. 
Estamos arriba del .75, pero tenemos que revisar el error medio cuadratico para poder llegar a una conclusion. 

RANDOM FOREST CLASSIFER MODEL.
---
Trabajaremos en esta seccion con el modelo bosque aleatorio, utilizando de igualforma el random state de 54321 y un numero de arboles de 3. 
Despues revisar con diferentes valores de numero de arboles para comparar que valor de n_estimators nos da una mejor exactitud. 

In [9]:
#modelo de clasificacion de bosque
model_forest = RandomForestClassifier(random_state= 54321 , n_estimators=3)
model_forest.fit(features_train , target_train)
valid_forest_prediction = model_forest.predict(features_valid)
#Predicciones del modelo de bosque 
print('Exactitud del conjunto de validacion:', model_forest.score(features_valid , target_valid))

Exactitud del conjunto de validacion: 0.7487562189054726


In [10]:
best_score_forest = 0
best_est_forest = 0
for est in range (1,11):
    model_forest = RandomForestClassifier(random_state=54321 , n_estimators= est)
    model_forest.fit(features_train , target_train)
    print('n_estimators = ' , est , ': ' , end= '')
    score_forest = model_forest.score(features_valid , target_valid)
    print('Con la exactitud',score_forest)
    if best_score_forest < score_forest:
        best_score_forest = score_forest
        best_est_forest = est

print('\nLa mejor exactitud es de {} con (n_estimators = {})'.format(best_score_forest,best_est_forest))

n_estimators =  1 : Con la exactitud 0.6878109452736318
n_estimators =  2 : Con la exactitud 0.7562189054726368
n_estimators =  3 : Con la exactitud 0.7487562189054726
n_estimators =  4 : Con la exactitud 0.7674129353233831
n_estimators =  5 : Con la exactitud 0.7574626865671642
n_estimators =  6 : Con la exactitud 0.7723880597014925
n_estimators =  7 : Con la exactitud 0.7786069651741293
n_estimators =  8 : Con la exactitud 0.7761194029850746
n_estimators =  9 : Con la exactitud 0.7736318407960199
n_estimators =  10 : Con la exactitud 0.7786069651741293

La mejor exactitud es de 0.7786069651741293 con (n_estimators = 7)


El modelo con n_estimators de 7 tuvo la mejor exactitud, 0.778.
Podemos norar que despues del estimado 6 ya no hay una cambio tan representativo en la extitud. Es más, el estimado 10 tuvo la misma exatitud que el estimado 7.


REGRESION LOGISTICA.
---
Detro de la regresión logistica igualmente utilizaremos el random state de 54321- 
Calificaremos la exactitud utilizando el conjunto de valores tanto de entrenamiento como de validacion. 

In [11]:
model_regression= LogisticRegression(random_state= 54321 , solver='liblinear')
model_regression.fit(features_train,target_train)
score_train_regression = model_regression.score(features_train,target_train)
score_valid_regression = model_regression.score(features_valid,target_valid)

print('La exactitud del modelo de regresion logistica en el conjunto de entrenamiento', score_train_regression)
print('La exactitud del modelo de regresion logistica en el conjunto de validacion', score_valid_regression)

La exactitud del modelo de regresion logistica en el conjunto de entrenamiento 0.7178423236514523
La exactitud del modelo de regresion logistica en el conjunto de validacion 0.6940298507462687


Podemos ver una exactitud muy baja de en este modelo, sin embargo, revisemos el error cuadratico de los modelos para poder eleguir uno de manera correcta. 

ERROR CUADRATICO MEDIO
---

Vamos a revisar el error medio cuadratico (EMC) de cada uno de los modelos para elegir el mejor. 

In [12]:
# Declaracion para el RECM. 
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']
features_train_ecm , features_valid_ecm , target_train_ecm, target_valid_ecm = train_test_split(features, target,test_size=0.25,random_state=54321)
print(features_train_ecm.shape)
print(features_valid_ecm.shape)
print(target_train_ecm.shape)
print(target_valid_ecm.shape)


(2410, 4)
(804, 4)
(2410,)
(804,)


In [13]:
#Prueba de cordura
predictions_cordura = pd.Series(target.mean(),index=target.index)
mse = mean_squared_error(target,predictions_cordura)
rmse = mse ** 0.5
print('El RECM en base al promedio de los valores de target es -> ',rmse)

El RECM en base al promedio de los valores de target es ->  0.46102797293043907


In [14]:
#DECISION TREE REGRESSOR. RECM
best_RECM_tree = 10000
best_depth = 0
for depth in range(1,11):
    model_tree_regressor = DecisionTreeRegressor(random_state=54321 , max_depth= depth)
    model_tree_regressor.fit(features_train_ecm , target_train_ecm)
    valid_tree_prediction_ecm = model_tree_regressor.predict(features_valid_ecm)
    print('max_depth = ' , depth , ': ' , end= '')
    RECM_tree = mean_squared_error(target_valid_ecm, valid_tree_prediction_ecm) ** 0.5
    print('RECM',RECM_tree)
    if RECM_tree < best_RECM_tree:
        best_model_tree = model_tree_regressor
        best_RECM_tree = RECM_tree

print('\nThe model with the best RECM was:\n',best_model_tree,'\nwith a RECM of : ' , best_RECM_tree)


max_depth =  1 : RECM 0.44098405178305866
max_depth =  2 : RECM 0.41935202725776616
max_depth =  3 : RECM 0.40919632941340167
max_depth =  4 : RECM 0.404940364204513
max_depth =  5 : RECM 0.40033089897845153
max_depth =  6 : RECM 0.39920585182877466
max_depth =  7 : RECM 0.4051436635073566
max_depth =  8 : RECM 0.4038884724633892
max_depth =  9 : RECM 0.4063362159345581
max_depth =  10 : RECM 0.4107715233752954

The model with the best RECM was:
 DecisionTreeRegressor(max_depth=6, random_state=54321) 
with a RECM of :  0.39920585182877466


Al analizar el Error Medio Cuadratico del modelo del árbol de desición de regresión, el modelo que mejor responde es el que tiene la profundidad de 6, con un 0.399 de RECM.
Este resultado es mejor que el RECM en la prueba de cordura.  

In [15]:
#Bosque aleatorio de regresion. RECM.
best_RECM_forest = 10000
best_est_forest = 0
best_depth_forest = 0
for est in range (10,30,10):
    print()
    for depth in range(1 , 11):
        model_forest_regressor = RandomForestRegressor(random_state=54321 , n_estimators= est , max_depth=depth)
        model_forest_regressor.fit(features_train_ecm , target_train_ecm)
        valid_forest_prediction_ecm = model_forest_regressor.predict(features_valid_ecm)
        RECM_forest = mean_squared_error(target_valid_ecm, valid_forest_prediction_ecm)**0.5
        print('El RECM es',RECM_forest, 'con el numero de arboles de',est,'y una profundidad de : ',depth)
        if RECM_forest < best_RECM_forest:
            best_RECM_forest = RECM_forest
            best_est_forest = est
            best_model_forest = model_forest_regressor
            best_max_depth_forest = depth
         

print('\nEl mejor RECM {} se obtuvo con (n_estimators = {}) y (max_depth = {})'.format(best_RECM_forest,best_est_forest,best_max_depth_forest))


El RECM es 0.43030643644756944 con el numero de arboles de 10 y una profundidad de :  1
El RECM es 0.4159120698914129 con el numero de arboles de 10 y una profundidad de :  2
El RECM es 0.4076383317865406 con el numero de arboles de 10 y una profundidad de :  3
El RECM es 0.3987678217072082 con el numero de arboles de 10 y una profundidad de :  4
El RECM es 0.39395586231233054 con el numero de arboles de 10 y una profundidad de :  5
El RECM es 0.3894803800929814 con el numero de arboles de 10 y una profundidad de :  6
El RECM es 0.3892000351356857 con el numero de arboles de 10 y una profundidad de :  7
El RECM es 0.3851956570606119 con el numero de arboles de 10 y una profundidad de :  8
El RECM es 0.3869745634090257 con el numero de arboles de 10 y una profundidad de :  9
El RECM es 0.3876881630086166 con el numero de arboles de 10 y una profundidad de :  10

El RECM es 0.43162530291565276 con el numero de arboles de 20 y una profundidad de :  1
El RECM es 0.4150566324763324 con el 

Despues de hacer varias pruebas de diferentes rangos, se puede ver  que para el modelo Bosque aleatorio los mejores hipervalores son (n_estimators = 20) y (max_depth = 9), con un 0.3838 de RECM. 
Podemos ver que despues del .38 ya no hay tanta mejora, pero al varias el sistema podemos ver una pequeña variacion que nos puede beneficiar. 
Este modelo tuvo una mejora en comparacion con la prueba de cordura y el modelo arbol de desicion. 

In [16]:
#Modelo de Regresion Lineal 
model_linel = LinearRegression()
model_linel.fit(features_train_ecm,target_train_ecm)
prediction_lineal_valid_ecm = model_linel.predict(features_valid_ecm)
RECM_linel = mean_squared_error(target_valid_ecm,prediction_lineal_valid_ecm)**0.5

print('RECM del modelo de regresion lineal en el conjunto de validacion',RECM_linel)

RECM del modelo de regresion lineal en el conjunto de validacion 0.45504947446604305


De nuevo, podemos ver que no es el mejor modelo en comparacion a los otros 2, esta por debajo de la prueba de cordura.
Sin embargo, el RECM es muy alto en comparacion al modelo bosque aleatorio. 

SELECCIÓN DE MODELO FINAL Y CONCLUSIONES
---

De acurdo con las pruebas anteriores, el mejor modelo a trabajar para determinar que plan es el utilizado y cual es el mas recomendable dependiendo el uso de datos, llamadas y mensajes, es el Bosque Aleatorio. 
Esto es por su buena exactitud utilizando un numero de arboles de 7 y al obtener el valor de RECM mas bajo podemos concluir que este modelo nos da respuestas con un rango de error mas pequeño y dentro del margen de un 75% de exactitud. 

In [19]:
final_features = features = df.drop(['is_ultra'],axis=1)
final_target = target = df['is_ultra']
final_model = RandomForestRegressor(random_state=54321,n_estimators=7)
final_model.fit(final_features,final_target)

predictions_final = final_model.predict(final_features)
recm_final = mean_squared_error(final_target,predictions_final)** 0.5
print('El RECM obtenido con el modelo de bosque aleatorio con 7 arboles es:',recm_final)

El RECM obtenido con el modelo de bosque aleatorio con 7 arboles es: 0.1781998979717685
